In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages


In [2]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [3]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\n## SESSION INTENT\nThe user is seeking information about Lunapolis, including its capital, weather, population of cheese miners, and the status of a potential strike by the cheese miners' union.\n\n## SUMMARY\n- The capital of the moon is identified as Lunapolis.\n- The current weather in Lunapolis is clear skies, with temperatures ranging from a high of 120C to a low of -100C.\n- There are 100,000 cheese miners living in Lunapolis.\n- The cheese miners' union is likely to strike due to dissatisfaction with the new president.\n\n## ARTIFACTS\nNone\n\n## NEXT STEPS\nNone", additional_kwargs={'lc_source': 'summarization'}, response_metadata={}, id='3c3e4d5b-ac33-4430-812c-cb68fd7e0679'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='cc1f06f0-a70b-4edb-89a9-602c37ba91af'),
     

In [4]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

## SESSION INTENT
The user is seeking information about Lunapolis, including its capital, weather, population of cheese miners, and the status of a potential strike by the cheese miners' union.

## SUMMARY
- The capital of the moon is identified as Lunapolis.
- The current weather in Lunapolis is clear skies, with temperatures ranging from a high of 120C to a low of -100C.
- There are 100,000 cheese miners living in Lunapolis.
- The cheese miners' union is likely to strike due to dissatisfaction with the new president.

## ARTIFACTS
None

## NEXT STEPS
None


## Trim/delete messages


In [6]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}


# @before_agent
# @after_agent
# once, before or after each run

# @before_model
# @after_model
# multiple times, before or after each model call

In [7]:
agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [8]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='654214b5-6ed3-4561-99a4-7ec6008a3d31'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='d6b6c130-2aac-4134-8080-99c8414efb41', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='283c24eb-3cee-4875-8336-e5c48c3b6699'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='599a1f6d-0ade-4bf9-9c5f-c601350f6eab', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='d98d3bdd-de87-43be-bf80-64e36e08f8f3'),
              AIMessage(content='I can’t read your device’s actual temperature from here, and i

In [9]:
print(response["messages"][-1].content)

I can’t read your device’s actual temperature from here, and if it won’t power on you won’t be able to see an internal temp readout anyway. If you want to check temps in the future (when it’s on), you’d use software or the BIOS/UEFI, depending on your device.

Right now, since it won’t turn on, here are steps to troubleshoot power and possible overheating-related shutdowns:

- Try a different power outlet and a known-good charger/cable that matches the device’s wattage.
- For laptops, if the battery is removable, reseat or remove the battery, then plug in power and try to start. If not removable, do a power reset: hold the power button for 15–20 seconds with the charger disconnected, then reconnect and try again.
- Check for any indicator lights or beeps. Note the color, pattern, or number of beeps, and tell me what you see.
- Check for obvious signs of damage or overheating: a very hot device, a swollen battery, a burnt smell, or blocked vents. If you see any of these, stop using it a